## 424. Longest Repeating Character Replacement
- Description:
  <blockquote>
  You are given a string `s` and an integer `k`. You can choose any character of the string and change it to any other uppercase English character. You can perform this operation at most `k` times.

  Return _the length of the longest substring containing the same letter you can get after performing the above operations_.

  **Example 1:**

  ```
  Input: s = "ABAB", k = 2
  Output: 4
  Explanation: Replace the two 'A's with two 'B's or vice versa.

  ```

  **Example 2:**

  ```
  Input: s = "AABABBA", k = 1
  Output: 4
  Explanation: Replace the one 'A' in the middle with 'B' and form "AABBBBA".
  The substring "BBBB" has the longest repeating letters, which is 4.
  There may exists other ways to achieve this answer too.
  ```

  **Constraints:**

  -   `1 <= s.length <= 10<sup>5</sup>`
  -   `s` consists of only uppercase English letters.
  -   `0 <= k <= s.length`
  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/longest-repeating-character-replacement/description/)

- Topics: Problem_topic

- Difficulty: Medium

- Resources: example_resource_URL

### Solution 1, Optimized sliding window
Solution description

Time complexity: O(n)
Space complexity: O(m) ~ O(1)

Where n is the length of the string and m is the total number of unique characters in the string, which for engligh alphabets is 26, so it is O(1).

You don’t reset or re-scan substrings like in the brute-force approach.
Instead, you slide the window dynamically, always maintaining the valid range.

Key Insight: For any substring to be valid, the number of characters that need to be replaced equals:
window_size - most_frequent_character_count
If this value ≤ k, we can make all characters in the window the same.


Now, given a substring, we want to find out if it can be converted into a string with the same letters. Because we are allowed only k operations to do so, we would want to minimize the number of operations.

We can divide all the characters of the string s into two groups - fixed letters and the letters that will be changed. Fixed letters remain unchanged. The rest of the letters would be substituted by fixed letters. To keep the number of substitutions minimum, the number of fixed letters must be maximum.

So, we find the character target, which occurs with the maximum frequency in the string. All other characters can now be replaced with target. If the count of other characters is less than or equal to k, then this substring fulfills the condition given in the question. We'd call it a valid substring.

In [ ]:
from collections import defaultdict


class Solution:
    def characterReplacement(self, s: str, k: int) -> int:
        longest_substring_length = 0
        left = 0
        char_max_freq = 0
        char_freq_map = defaultdict(int)
        
        for right in range(len(s)):
            char_freq_map[s[right]] += 1
            char_max_freq = max(char_max_freq, char_freq_map[s[right]])

            # If characters needing replacement > k, shrink from left
            while (right - left + 1) - char_max_freq > k:
                char_freq_map[s[left]] -= 1
                left += 1
            
            longest_substring_length = max(longest_substring_length, right - left + 1)

        return longest_substring_length

### Solution 2, Further optimized sliding window with Non-Shrinking Window
Once we find a window of size n, we only care about finding larger windows. So instead of shrinking, we just slide the window right maintaining the same size until we find a bigger valid window.

in the second solution, max_frequency becomes a "maximum frequency ever seen" rather than the current window's max frequency. This is fine because:
If current window is invalid with old max_frequency, it would still be invalid with the true current max
We're only looking for windows larger than what we've found, so this optimization doesn't miss the answer

- Time Complexity: O(N)
- Space Complexity: O(M), m unique characters in the string, Considering uppercase English letters only, m=26.

In [ ]:
class Solution:
    def characterReplacement(self, s: str, k: int) -> int:
        left = 0
        frequency_map = {}
        max_frequency = 0
        longest_substring_length = 0
        for right in range(len(s)):
            frequency_map[s[right]] = frequency_map.get(s[right], 0) + 1

            # the maximum frequency we have seen in any window yet, AKA Maximum Frequency Ever Seen
            """
            Why this works:
            If we had frequency f in a previous window, and that window was valid, we want to find windows larger than that
            If current window is invalid with old max_frequency, it would definitely be invalid with the true current max frequency
            We never need to decrease max_frequency because we're only looking for bigger windows
            """
            
            max_frequency = max(max_frequency, frequency_map[s[right]])

            # move the start pointer towards right if the current
            # window is invalid
            is_valid = ((right - left + 1) - max_frequency <= k)
            
            # Handle Invalid Window (Slide, Don't Shrink)
            # Move left pointer right by 1
            # Move right pointer right by 1 (in next iteration)
            # Net effect: Window slides right, same size
            if not is_valid:
                frequency_map[s[left]] -= 1
                left += 1

            # the window is valid at this point, store length
            # size of the window never decreases
            longest_substring_length = right + 1 - left

        return longest_substring_length